## Sobre la base de datos

Está relacionada al trabajo:
[CalcAMP: A New Machine Learning Model for the Accurate Prediction of Antimicrobial Activity of Peptides](https://pmc.ncbi.nlm.nih.gov/articles/PMC10135148/)

Contiene registros positivos y negativos de antifúngicos en formato FASTA.

Usaremos la biblioteca `Biopython` que es muy útil para trabajar con secuencias biológicas.

In [1]:
%pip install biopython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 34.5 MB/s eta 0:00:00


In [2]:
from Bio import SeqIO
import pandas as pd

In [3]:
fasta_train = '/content/training_fungi.fasta'
fasta_test = '/content/Holdout_AFP.fasta'

## Secuencias entrenamiento

In [4]:
data = []
try:
    for record in SeqIO.parse(fasta_train, "fasta"):
        data.append({
            "ID": record.id,
            "Description": record.description,
            "Sequence": str(record.seq)
        })

    df_train = pd.DataFrame(data)
    display(df_train.head())

except FileNotFoundError:
    print(f"Error: El archivo no se encuentra en la ruta especificada: {fasta_train}")
except Exception as e:
    print(f"Ocurrió un error al parsear el archivo fasta: {e}")

,ID,Description,Sequence
0,Non-AFP_0001,Non-AFP_0001,SGTSEKERESERLLGVVNPLIKCFRSPCP
1,AFP_0001,AFP_0001,FVPWFSKFLKRIL
2,AFP_0002,AFP_0002,GFKMALKLLKKVL
3,AFP_0003,AFP_0003,SLLSLFRKLIT
4,Non-AFP_0002,Non-AFP_0002,GWGSIFKHIFKAGKFIHGAIQAHNDG


In [5]:
# Filter df_train for 'Non-AFP_' in the 'Description' column
df_train_neg = df_train[df_train['Description'].str.startswith('Non-AFP_')].copy()

# Filter df_train for 'AFP_' in the 'Description' column, ensuring it starts with 'AFP_'
df_train_pos = df_train[df_train['Description'].str.startswith('AFP_')].copy()

In [6]:
print("Training data - Non-AFP_:")
display(df_train_neg.head())

Training data - Non-AFP_:


,ID,Description,Sequence
0,Non-AFP_0001,Non-AFP_0001,SGTSEKERESERLLGVVNPLIKCFRSPCP
4,Non-AFP_0002,Non-AFP_0002,GWGSIFKHIFKAGKFIHGAIQAHNDG
7,Non-AFP_0003,Non-AFP_0003,FLPLIAGLAAKCAITKKC
8,Non-AFP_0004,Non-AFP_0004,SLWSSIKDMAAAAGRAALNAVNGILNP
9,Non-AFP_0005,Non-AFP_0005,RLWRRWRRWLR


In [7]:
print("\nTraining data - AFP_:")
display(df_train_pos.head())


Training data - AFP_:


,ID,Description,Sequence
1,AFP_0001,AFP_0001,FVPWFSKFLKRIL
2,AFP_0002,AFP_0002,GFKMALKLLKKVL
3,AFP_0003,AFP_0003,SLLSLFRKLIT
5,AFP_0004,AFP_0004,ALWKSILKNVGKAAGKAVLNAVTDMVNQ
6,AFP_0005,AFP_0005,GIINTLQKYYSRVRGGR


## Secuencia de prueba

In [8]:
data = []
try:
    for record in SeqIO.parse(fasta_test, "fasta"):
        data.append({
            "ID": record.id,
            "Description": record.description,
            "Sequence": str(record.seq)
        })

    df_test = pd.DataFrame(data)
    display(df_test.head())

except FileNotFoundError:
    print(f"Error: El archivo no se encuentra en la ruta especificada: {fasta_test}")
except Exception as e:
    print(f"Ocurrió un error al parsear el archivo fasta: {e}")

,ID,Description,Sequence
0,Non-AFP_00,Non-AFP_00,GLWSTIKNVGKEAAIAAGKAVLGSL
1,Non-AFP_01,Non-AFP_01,VKRFKKFFRKLKKSVKKL
2,Non-AFP_02,Non-AFP_02,GAKLAKKQVRALGKFFSF
3,Non-AFP_03,Non-AFP_03,HFLGKLVNLAKKIL
4,Non-AFP_04,Non-AFP_04,WRWRWR


In [9]:
# Filter df_train for 'Non-AFP_' in the 'Description' column
df_test_neg = df_test[df_test['Description'].str.startswith('Non-AFP_')].copy()

# Filter df_train for 'AFP_' in the 'Description' column, ensuring it starts with 'AFP_'
df_test_pos = df_test[df_test['Description'].str.startswith('AFP_')].copy()

In [10]:
print("Training data negative - Non-AFP_:")
display(df_train_neg.head())

Training data negative - Non-AFP_:


,ID,Description,Sequence
0,Non-AFP_0001,Non-AFP_0001,SGTSEKERESERLLGVVNPLIKCFRSPCP
4,Non-AFP_0002,Non-AFP_0002,GWGSIFKHIFKAGKFIHGAIQAHNDG
7,Non-AFP_0003,Non-AFP_0003,FLPLIAGLAAKCAITKKC
8,Non-AFP_0004,Non-AFP_0004,SLWSSIKDMAAAAGRAALNAVNGILNP
9,Non-AFP_0005,Non-AFP_0005,RLWRRWRRWLR


In [11]:
print("\nTest data - AFP_:")
display(df_train_pos.head())


Test data - AFP_:


,ID,Description,Sequence
1,AFP_0001,AFP_0001,FVPWFSKFLKRIL
2,AFP_0002,AFP_0002,GFKMALKLLKKVL
3,AFP_0003,AFP_0003,SLLSLFRKLIT
5,AFP_0004,AFP_0004,ALWKSILKNVGKAAGKAVLNAVTDMVNQ
6,AFP_0005,AFP_0005,GIINTLQKYYSRVRGGR


## Concatenar ambas secuencias

In [12]:
# Seleccionar las secuencias negativas
train_pos = df_train_pos[['Sequence']]
test_pos = df_test_pos[['Sequence']]

# Agregar una columna de label (si tiene actividad antifúngica o no)
train_pos['label'] = 1
test_pos['label'] = 1

/tmp/ipython-input-2009855227.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_pos['label'] = 1
/tmp/ipython-input-2009855227.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_pos['label'] = 1


In [13]:
# Seleccionar las secuencias negativas
train_neg = df_train_neg[['Sequence']]
test_neg = df_test_neg[['Sequence']]

# Agregar una columna de label (si tiene actividad antifúngica o no)
train_neg['label'] = 0
test_neg['label'] = 0

/tmp/ipython-input-4186712105.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_neg['label'] = 0
/tmp/ipython-input-4186712105.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_neg['label'] = 0


In [14]:
# Concatenar los dataframes
df_combined = pd.concat([train_pos, train_neg, test_pos, test_neg])

In [15]:
df_combined.head()

,Sequence,label
1,FVPWFSKFLKRIL,1
2,GFKMALKLLKKVL,1
3,SLLSLFRKLIT,1
5,ALWKSILKNVGKAAGKAVLNAVTDMVNQ,1
6,GIINTLQKYYSRVRGGR,1


In [17]:
df_combined.shape

(2248, 2)

In [18]:
df_combined = df_combined.rename(columns={'Sequence': 'sequence'})
display(df_combined.head())

,sequence,label
1,FVPWFSKFLKRIL,1
2,GFKMALKLLKKVL,1
3,SLLSLFRKLIT,1
5,ALWKSILKNVGKAAGKAVLNAVTDMVNQ,1
6,GIINTLQKYYSRVRGGR,1


## Verificar duplicados

In [20]:
df_combined["sequence"].unique().shape

(2248,)

In [22]:
df_combined = df_combined.drop_duplicates().copy()

In [23]:
df_combined.shape

(2248, 2)

## Verificar canónicos

In [24]:
# Definir aminoácidos canónicos
aminoacidos = set("ACDEFGHIKLMNPQRSTVWY")

In [25]:
# Verificar cuáles secuencias válidas
df_combined["is_valid"] = [set(seq).issubset(aminoacidos) for seq in df_combined["sequence"]]

In [26]:
no_validas = df_combined[~df_combined["is_valid"]]
print("Secuencias inválidas:", len(no_validas))

Secuencias inválidas: 0


In [27]:
df_validas = df_combined[df_combined["is_valid"]].drop(columns="is_valid")
print("Secuencias válidas restantes:", len(df_validas))

Secuencias válidas restantes: 2248


In [28]:
display(df_validas)

,sequence,label
1,FVPWFSKFLKRIL,1
2,GFKMALKLLKKVL,1
3,SLLSLFRKLIT,1
5,ALWKSILKNVGKAAGKAVLNAVTDMVNQ,1
6,GIINTLQKYYSRVRGGR,1
...,...,...
25,FGTGVKRKAQCDLKSK,0
26,FAKKLLAKALKL,0
27,GIVEQCCASVCSLYQLENYCN,0
28,SALVGCWTKSYPPKPCFGR,0


In [29]:
df_validas.to_csv('CalcAMP_labeled.csv', index=False, header=True)

## Obtener metadatos

In [30]:
import json

In [31]:
def export_json(path_to_export, data_to_export):
    with open(path_to_export, 'w') as doc_export:
        json.dump(
            data_to_export,
            doc_export,
            indent=4,
            default=str,
            ensure_ascii=False)

In [32]:
def create_metada_with_multiple_values(df_metada_filter, full_df):
    dict_metadata = {}

    for column in df_metada_filter.columns:
        values = df_metada_filter[column].unique().tolist()

        if len(values)>1:
            values = [str(value) for value in values]
            values = ";".join(values)
            dict_metadata.update({column:values})
        else:
            dict_metadata.update({column:values[0]})

    dict_metadata.update({
        "number_of_sequences" : len(full_df)
    })

    return dict_metadata

In [33]:
def read_metadata(path_data, name_source):
    df_metada = pd.read_excel(path_data)
    df_metada_filter = df_metada[df_metada["name source"] == name_source]
    df_metada_filter = df_metada_filter[['type source',
                                         'estatico-dinamico',
                                         'licencia',
                                         'año de publicación',
                                         'fecha ultima actualizacion',
                                         'download date',
                                         'formato',
                                         'peptide property',
                                         'informacion del dataset',
                                         'unidad de medida',
                                         'Construccion de dataset negativos',
                                         'repositorio o servidor',
                                         'Publicacion',]]
    return df_metada_filter

In [35]:
df_metada_filter = read_metadata("/content/description_raw_data.xlsx", "CalcAMP")

In [36]:
dict_metadata = create_metada_with_multiple_values(df_metada_filter, df_validas)

In [37]:
export_json("metadata.json", dict_metadata)